In [2]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [3]:
engine.table_names()

AttributeError: 'Engine' object has no attribute 'table_names'

In [4]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.34


Per this [stackoverflow thread](https://stackoverflow.com/questions/76100113/i-get-the-following-error-running-sqlalchemy-in-replit-attributeerror-engine), the .table_names() method was deprecated since version 1.4!

In [5]:
from sqlalchemy import inspect
insp = inspect(engine)

In [6]:
insp.get_table_names()

['Bookings', 'Facilities', 'Members']

In [7]:
conn = engine.connect()

In [8]:
rs = conn.execute("SELECT * FROM Facilities")

ObjectNotExecutableError: Not an executable object: 'SELECT * FROM Facilities'

As it so happens, ```Connection.execute()``` has also changed, per the [sqlalchemy 2.0 documentation](https://docs.sqlalchemy.org/en/20/changelog/migration_20.html).  Instead of directly passing the SQL query as a string, it must be first converted into their own ```sqlalchemy.sql.elements.TextClause``` object, using the ```sqlalchemy.text()``` method:

In [9]:
from sqlalchemy import text
rs = conn.execute(text("SELECT * FROM Facilities"))

In [10]:
type(text("SELECT * FROM Facilities"))

sqlalchemy.sql.elements.TextClause

In [11]:
import pandas as pd
df = pd.DataFrame(rs.fetchall())
# df.columns = rs.keys()

In [12]:
print(df)

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   
5      5   Massage Room 2         9.9       80.0           4000   
6      6     Squash Court         3.5       17.5           5000   
7      7    Snooker Table         0.0        5.0            450   
8      8       Pool Table         0.0        5.0            400   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  
5                3000  
6                  80  
7                  15  
8                  15  


In [13]:
conn.close()

In [26]:
def run_sql_query(query=None):
    if not query:
        query=input()
    with engine.connect() as conn:
        rs = conn.execute(text(query))
        df = pd.DataFrame(rs.fetchall())
    return df

In [27]:
run_sql_query()

 SELECT * FROM Facilities


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [28]:
f = run_sql_query("SELECT * FROM Facilities")
b = run_sql_query("SELECT * FROM Bookings")
m = run_sql_query("SELECT * FROM Members")

In [29]:
f.columns

Index(['facid', 'name', 'membercost', 'guestcost', 'initialoutlay',
       'monthlymaintenance'],
      dtype='object')

In [30]:
b.columns

Index(['bookid', 'facid', 'memid', 'starttime', 'slots'], dtype='object')

In [31]:
m.columns

Index(['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone',
       'recommendedby', 'joindate'],
      dtype='object')

In [33]:
[ col for col in f.columns if col in b.columns ]

['facid']

In [34]:
[ col for col in f.columns if col in m.columns ]

[]

In [35]:
[ col for col in b.columns if col in m.columns ]

['memid']

#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

To get revenue for a given booking, you multiply slots * cost (member or guest) for that booking.

1) Combine bookings with members and calculate costs for each one
2) Combine with facilities in order to group by facility
3) Sum up the revenue of all bookings for each facility
4) Filter for facility revenue < 1000

In [38]:
query="""
SELECT *
FROM Members as m
INNER JOIN Bookings as b
    ON m.memid = b.memid
"""
run_sql_query(query)

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate,bookid,facid,memid,starttime,slots
0,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,0,3,1,2012-07-03 11:00:00,2
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,1,4,1,2012-07-03 08:00:00,2
2,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00,2,6,0,2012-07-03 18:00:00,2
3,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,3,7,1,2012-07-03 19:00:00,2
4,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05,4,8,1,2012-07-03 10:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4038,29,Worthington-Smyth,Henry,"55 Jagbi Way, North Reading",97676,(855) 894-3758,2,2012-09-17 12:27:15,4038,8,29,2012-09-30 16:30:00,2
4039,29,Worthington-Smyth,Henry,"55 Jagbi Way, North Reading",97676,(855) 894-3758,2,2012-09-17 12:27:15,4039,8,29,2012-09-30 18:00:00,1
4040,21,Mackenzie,Anna,"64 Perkington Lane, Reading",64577,(822) 661-2898,1,2012-08-26 09:32:05,4040,8,21,2012-09-30 18:30:00,1
4041,16,Baker,Timothy,"329 James Street, Reading",58393,833-941-0824,13,2012-08-15 10:34:25,4041,8,16,2012-09-30 19:00:00,1


In [39]:
query="""
SELECT 
    m.memid,
    facid,
    bookid,
    slots
    
FROM Members as m
INNER JOIN Bookings as b
    ON m.memid = b.memid
"""
run_sql_query(query)

,memid,facid,bookid,slots
0,1,3,0,2
1,1,4,1,2
2,0,6,2,2
3,1,7,3,2
4,1,8,4,1
...,...,...,...,...
4038,29,8,4038,2
4039,29,8,4039,1
4040,21,8,4040,1
4041,16,8,4041,1


In [55]:
query="""
SELECT facility, SUM(cost) AS total_revenue

FROM (
    SELECT 
        f.name AS facility,
        (CASE WHEN m.memid = 0 
        THEN slots * guestcost
        ELSE slots * membercost END) AS cost
        
    FROM Members AS m
    INNER JOIN Bookings AS b
        ON m.memid = b.memid
    
    INNER JOIN Facilities as f
        ON f.facid = b.facid
) AS subq

GROUP BY facility
HAVING SUM(cost) < 1000
ORDER BY total_revenue

"""
run_sql_query(query)

,facility,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [67]:
m.columns

Index(['memid', 'surname', 'firstname', 'address', 'zipcode', 'telephone',
       'recommendedby', 'joindate'],
      dtype='object')

In [70]:
query="""

SELECT 
    member_name,
    recommender_name

FROM(
    SELECT 
        recommendedby AS id,
        CONCAT_WS(',',surname,firstname) AS member_name
    FROM Members;
) AS main

INNER JOIN(
    SELECT
        memid AS id,
        CONCAT_WS(',',surname,firstname) AS recommender_name
    FROM Members;
) AS recs

ON main.id = recs.id
        
"""
run_sql_query(query)

OperationalError: (sqlite3.OperationalError) near ";": syntax error
[SQL: 

SELECT 
    member_name,
    recommender_name

FROM(
    SELECT 
        recommendedby AS id,
        CONCAT_WS(',',surname,firstname) AS member_name
    FROM Members;
) AS main

INNER JOIN(
    SELECT
        memid AS id,
        CONCAT_WS(',',surname,firstname) AS recommender_name
    FROM Members;
) AS recs

ON main.id = recs.id
        
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [85]:
query="""

    SELECT 
        recommendedby AS id,
        CONCAT_WS(',',surname,firstname) AS member_name
    FROM Members
        
"""
data=run_sql_query(query)
data

,id,member_name
0,,"GUEST,GUEST"
1,,"Smith,Darren"
2,,"Smith,Tracy"
3,,"Rownam,Tim"
4,1,"Joplette,Janice"
5,1,"Butters,Gerald"
6,,"Tracy,Burton"
7,4,"Dare,Nancy"
8,3,"Boothe,Tim"
9,6,"Stibbons,Ponder"


In [93]:
list(data['id'])[0]

''

In [94]:
query="""

    SELECT 
        recommendedby AS id,
        CONCAT_WS(',',surname,firstname) AS member_name
    FROM Members
    WHERE recommendedby != ''
        
"""
run_sql_query(query)

,id,member_name
0,1,"Joplette,Janice"
1,1,"Butters,Gerald"
2,4,"Dare,Nancy"
3,3,"Boothe,Tim"
4,6,"Stibbons,Ponder"
5,1,"Owen,Charles"
6,4,"Jones,David"
7,9,"Baker,Anne"
8,1,"Smith,Jack"
9,9,"Bader,Florence"


In [100]:
query="""

SELECT 
    member_name,
    recommender_name

FROM(
    SELECT 
        recommendedby AS id,
        CONCAT_WS(', ',surname,firstname) AS member_name
    FROM Members
    WHERE recommendedby != ''
) AS main

INNER JOIN(
    SELECT
        memid AS id,
        CONCAT_WS(', ',surname,firstname) AS recommender_name
    FROM Members
    WHERE recommendedby != ''
) AS recs

ON main.id = recs.id
ORDER BY member_name, recommender_name ASC
        
"""
run_sql_query(query)

,member_name,recommender_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Coplin, Joan","Baker, Timothy"
3,"Dare, Nancy","Joplette, Janice"
4,"Genting, Matthew","Butters, Gerald"
5,"Hunt, John","Purview, Millicent"
6,"Jones, David","Joplette, Janice"
7,"Jones, Douglas","Jones, David"
8,"Rumney, Henrietta","Genting, Matthew"
9,"Sarwin, Ramnaresh","Bader, Florence"


In [109]:
print(f"\nFacilities\n")
query="""

SELECT *
FROM Facilities
LIMIT 3
        
"""
run_sql_query(query)


Facilities



,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5,25.0,10000,200
1,1,Tennis Court 2,5,25.0,8000,200
2,2,Badminton Court,0,15.5,4000,50


In [115]:
print(f"\n===MEMBERS===\n")
query="""

SELECT *
FROM Members
LIMIT 3
        
"""
run_sql_query(query)


===MEMBERS===



,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23


In [113]:
print(f"\n===BOOKINGS===\n")
query="""

SELECT *
FROM Bookings
LIMIT 3
        
"""
run_sql_query(query)


===BOOKINGS===



,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


In [129]:
query="""
SELECT
    f.facility,
    COUNT(m.memid) AS member_bookings

FROM (SELECT memid FROM Members WHERE memid != 0) AS m

INNER JOIN (SELECT bookid,facid,memid FROM Bookings) AS b
    ON m.memid = b.memid

INNER JOIN (SELECT facid, name AS facility FROM Facilities) AS f
    ON f.facid = b.facid

GROUP BY f.facility
"""
run_sql_query(query)

,facility,member_bookings
0,Badminton Court,344
1,Massage Room 1,421
2,Massage Room 2,27
3,Pool Table,783
4,Snooker Table,421
5,Squash Court,195
6,Table Tennis,385
7,Tennis Court 1,308
8,Tennis Court 2,276


In [144]:
query="""
SELECT
    facility,
    STRFTIME('%m', b.starttime) AS month,
    COUNT(m.memid) AS member_bookings

FROM (SELECT memid FROM Members WHERE memid != 0) AS m

INNER JOIN (SELECT bookid,facid,memid, starttime FROM Bookings) AS b
    ON m.memid = b.memid

INNER JOIN (SELECT facid, name AS facility FROM Facilities) AS f
    ON f.facid = b.facid

GROUP BY facility,STRFTIME('%m', b.starttime)
"""
run_sql_query(query)

,facility,month,member_bookings
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
